# Testes de acessibilidade

## Introdução

Este notebook conduz testes automatizados de acessibilidade nos portais mapeados no Censo Querido Diário.

Segundo o último Censo Demográfico, há 46 milhões de pessoas no Brasil com algum tipo de deficiência - o equivalente a quase 24% da população<sup>[1](https://educa.ibge.gov.br/jovens/conheca-o-brasil/populacao/20551-pessoas-com-deficiencia.html)</sup>. Desde 2000, a promoção de mecanismos para garantir o acesso à informação desse segmento da população é uma obrigação do poder público prevista em lei<sup>[2](http://www.planalto.gov.br/ccivil_03/leis/l10098.htm)</sup>.

O objetivo desta análise é verificar se essa obrigação tem sido cumprida pelos municípios brasileiros, no que se refere ao acesso a informações contidas nos diários oficiais. Para isso, buscamos verificar se os portais que dão acesso aos  estão de acordo com os padrões da web para garantir o acesso por pessoas com deficiência.

---

<sup>1</sup>: *IBGE. Conheça o Brasil - População: Pessoas com Deficiência. S/D. Disponível em: <https://educa.ibge.gov.br/jovens/conheca-o-brasil/populacao/20551-pessoas-com-deficiencia.html>.*

<sup>2</sup>: *BRASIL. Lei Nº 10.098, de 19 de dezembro de 2000. 2000. Disponível em: <http://www.planalto.gov.br/ccivil_03/leis/l10098.htm>.*

## Dependências

Esta análise utiliza um driver de navegador (*geckodrive*) para simular o comportamento de uma pessoa navegando pelos portais dos diários oficiais.

Nas células a seguir, verificamos se o executável desse driver está localizado em algum lugar da raíz do repositório do Censo, no diretório `/notebooks` ou em algum lugar do seu caminho de busca padrão (`$PATH`). Caso não seja encontrado, o programa tentará baixar uma versão do Geckodriver adequada para o seu sistema.

Também são instalados os pacotes [`selenium`](https://pypi.org/project/selenium/) e [`axe-selenium-python`](https://pypi.org/project/axe-selenium-python/) para realizar os testes automatizados. Recomenda-se rodar este notebook em um ambiente virtual para que a instalação desses pacotes não gere conflitos com as dependências instaladas nos sistema.

In [173]:
!pip install -qq requests selenium axe-selenium-python wget 
!pip install -qq pandas scipy plotly nbformat>=4.2.0

In [2]:
import os
import platform
import shutil
import sys
from pathlib import Path

import wget
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.firefox.options import Options as FirefoxOptions


def download_gecko(version="v0.29.1") -> None:
    # get Gecko executable
    system = platform.system()
    architecture = (int(sys.maxsize > 2**32) + 1) * 32
    gecko_executable_url = (
        f"https://github.com/mozilla/geckodriver/releases/download/{version}/"
        f"geckodriver-{version}-"
    )
    if system == "Windows":
        gecko_executable_url += f"windows{architecture}.zip"
    elif system == "Linux":
        gecko_executable_url += f"linux{architecture}.tar.gz"
    elif system == "Darwin":
        gecko_executable_url += "macos"
        if system.architecture.startswith("arm"):
            gecko_executable_url += "-aarch64"
        gecko_executable_url += ".tar.gz"
    else:
        raise RuntimeError(
            f"No Geckodriver executable available for {system} {architecture}"
        )
    gecko_compressed = wget.download(gecko_executable_url)
    shutil.unpack_archive(gecko_compressed)


# check if geckodriver has been downloaded to the current working directory
driver_options = FirefoxOptions()
driver_options.headless = True
gecko_local_executable = os.path.join(os.getcwd(), "geckodriver")
if Path(gecko_local_executable).is_file():
    executable_path = gecko_local_executable
else:
    executable_path = None

# test creating a new driver; download Gecko if needed
try:
    driver = webdriver.Firefox(
        options=driver_options,
        executable_path=executable_path,
    )
except WebDriverException:
    download_gecko()
    executable_path = gecko_local_executable
    driver = webdriver.Firefox(
        options=driver_options,
        executable_path=executable_path,
    )
finally:
    driver.close()

## Requisitar dados do Censo Querido Diário

Nesta seção, baixamos os dados mais recentes do mapeamento de portais de diários oficiais do Censo Querido Diário. Em seguida, transformamos as diversas colunas com as URLs das fontes em uma única coluna (formato *longo*).

In [3]:
import pandas as pd

census_data = pd.read_csv("https://censo.ok.org.br/get-data/")
census_data

,municipio,IBGE,IBGE7,UF,regiao,populacao_2020,eh_capital,fonte_1,fonte_2,fonte_3,fonte_4,is_online,data_inicial,tipo_arquivo,validacao,navegacao,observacoes
0,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,https://abadiadosdourados.mg.gov.br/novo/index...,None,None,None,1,2013-12-30,PDF texto,True,None,"Desde outubro de 2017, passaram a ser publicad..."
1,Abadia de Goiás (GO),520005,5200050,GO,Região Centro-Oeste,8958,False,None,None,None,None,2,None,None,True,None,Atos publicados de forma avulsa no site da pre...
2,Abaetetuba (PA),150010,1500107,PA,Região Norte,159080,False,http://www.diariomunicipal.com.br/famep/pesqui...,None,None,None,1,2016-06-30,HTML,True,None,NaN
3,Abaré (BA),290020,2900207,BA,Região Nordeste,20347,False,https://sai.io.org.br/ba/abare/site/diariooficial,None,None,None,1,2007-01-09,PDF texto,True,None,None
4,Abaíra (BA),290010,2900108,BA,Região Nordeste,8710,False,http://diariooficial.portalgov.net.br/,None,None,None,1,2017-01-06,PDF imagem,True,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,Águas de Lindóia (SP),350050,3500501,SP,Região Sudeste,18808,False,https://imprensaoficialmunicipal.com.br/aguas_...,None,None,None,1,2020-01-31,PDF texto,True,None,NaN
593,Álvares Florence (SP),350120,3501202,SP,Região Sudeste,3647,False,https://imprensaoficialmunicipal.com.br/alvare...,None,None,None,1,2013-03-13,PDF texto,True,None,NaN
594,Álvares Machado (SP),350130,3501301,SP,Região Sudeste,24998,False,http://www.alvaresmachado.sp.gov.br/diariooficial,None,None,None,1,2018-06-04,PDF texto,True,None,NaN
595,Álvaro de Carvalho (SP),350140,3501400,SP,Região Sudeste,5274,False,https://imprensaoficialmunicipal.com.br/alvaro...,None,None,None,1,2018-04-04,PDF texto,True,None,NaN


In [2]:
portals_info = (
    census_data
    .melt(
        id_vars=[
            "municipio",
            "IBGE",
            "IBGE7",
            "UF",
            "regiao",
            "populacao_2020",
            "eh_capital",
        ],
        value_vars=[
            col for col in census_data.columns if col.startswith("fonte_")
        ],
        value_name="fonte",
        var_name="fonte_prioridade",
    )
    .assign(
        fonte_prioridade=lambda _: pd.to_numeric(_.fonte_prioridade.str[-1])
    )
    .replace(to_replace='None', value=pd.NA)
    .dropna(subset=["fonte"])
)

portals_info

,municipio,IBGE,IBGE7,UF,regiao,populacao_2020,eh_capital,fonte_prioridade,fonte
0,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...
2,Abaetetuba (PA),150010,1500107,PA,Região Norte,159080,False,1,http://www.diariomunicipal.com.br/famep/pesqui...
3,Abaré (BA),290020,2900207,BA,Região Nordeste,20347,False,1,https://sai.io.org.br/ba/abare/site/diariooficial
4,Abaíra (BA),290010,2900108,BA,Região Nordeste,8710,False,1,http://diariooficial.portalgov.net.br/
5,Abreu e Lima (PE),260005,2600054,PE,Região Nordeste,100346,False,1,http://www.diariomunicipal.com.br/amupe/
...,...,...,...,...,...,...,...,...,...
1364,Cocal do Sul (SC),420425,4204251,SC,Região Sul,16821,False,3,https://www.diariomunicipal.sc.gov.br/site/?r=...
1385,Cuiabá (MT),510340,5103403,MT,Região Centro-Oeste,618124,True,3,https://diariooficial.cuiaba.mt.gov.br/
1440,Guarabira (PB),250630,2506301,PB,Região Nordeste,59115,False,3,https://pmguarabira.wixsite.com/pmguarabira/di...
1611,Pará de Minas (MG),314710,3147105,MG,Região Sudeste,94808,False,3,https://transparencia.parademinas.mg.gov.br/


## Executar a avaliação automatizada dos portais

A seguir, iteramos sobre todos os portais mapeados no Censo Querido Diário, executando os testes automatizados disponibilizados pelo pacote [Axe](https://github.com/dequelabs/axe-core), e registrando as violações aos padrões web de acessibilidade detectadas.

**ATENÇÃO**: Podem ser necessárias algumas horas para realizar a avaliação de todos os portais mapeados no Censo.

In [68]:
import logging

from axe_selenium_python import Axe


logger = logging.getLogger()
logger.setLevel(logging.INFO)


def evaluate_a11y(url: str, driver: webdriver.Firefox) -> pd.DataFrame:
    """Performs an automated acessibility test in the given URL."""
    logger.info(f"Evaluating accessibility in portal <{url}>...")
    driver.get(url)
    axe = Axe(driver)
    # inject axe-core javascript into page.
    axe.inject()
    # run axe accessibility checks.
    results = axe.run()
    # convert into a DataFrame
    df_violations = pd.DataFrame(results["violations"])
    num_violations = len(portal_violations.index)
    num_violations_critical = sum(portal_violations["impact"] == "critical")
    num_violations_serious = sum(portal_violations["impact"] == "serious")
    num_violations_moderate = sum(portal_violations["impact"] == "moderate")
    num_violations_minor = sum(portal_violations["impact"] == "minor")
    logger.info(
        f"Found {num_violations} violations "
        f"({num_violations_critical} critical; "
        f"{num_violations_serious} serious; "
        f"{num_violations_moderate} moderate; "
        f"{num_violations_minor} minor)."
    )
    return df_violations

try:
    # create a new driver
    driver = webdriver.Firefox(
        options=driver_options,
        executable_path=executable_path,
    )
    # create a DataFrame to record violations across all portals
    all_violations = pd.DataFrame()
    # evaluate one portal at a time
    num_evaluations = 0
    portal_urls = portals_info["fonte"].unique()
    for url in portal_urls:
        try:
            num_evaluations += 1
            logger.info(
                f"Starting evaluation {num_evaluations} of {len(portal_urls)}."
            )
            portal_violations = evaluate_a11y(url=url, driver=driver)
            portal_violations["fonte"] = url
            all_violations = pd.concat(
                [all_violations, portal_violations],
                ignore_index=True,
            )
        except Exception as err:
            logger.error(f"Error while evaluating <{url}>: {err}")

finally:
    # finish driver
    driver.close()

tarting evaluation 535 of 618.
INFO:root:Evaluating accessibility in portal <https://ioes.dio.es.gov.br/caderno_municipios>...
INFO:root:Found 1 violations (1 critical; 0 serious; 0 moderate).
INFO:root:Starting evaluation 536 of 618.
INFO:root:Evaluating accessibility in portal <https://dom.pmvc.ba.gov.br/>...
INFO:root:Found 8 violations (3 critical; 2 serious; 3 moderate).
INFO:root:Starting evaluation 537 of 618.
INFO:root:Evaluating accessibility in portal <http://www.diariomunicipal.com.br/amupe/pesquisar?busca_avancada[entidadeUsuaria]=46229>...
INFO:root:Found 5 violations (1 critical; 1 serious; 2 moderate).
INFO:root:Starting evaluation 538 of 618.
INFO:root:Evaluating accessibility in portal <https://new.voltaredonda.rj.gov.br/8-interno/658-vr-destaque>...
INFO:root:Found 2 violations (0 critical; 1 serious; 1 moderate).
INFO:root:Starting evaluation 539 of 618.
INFO:root:Evaluating accessibility in portal <https://www.votorantim.sp.gov.br/portal/diario-oficial>...
INFO:root

In [69]:
portals_violations = portals_info.merge(
    all_violations,
    on="fonte",
    how="left",
)

portals_violations

,municipio,IBGE,IBGE7,UF,regiao,populacao_2020,eh_capital,fonte_prioridade,fonte,description,help,helpUrl,id,impact,nodes,tags
0,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures the contrast between foreground and ba...,Elements must have sufficient color contrast,https://dequeuniversity.com/rules/axe/3.1/colo...,color-contrast,serious,"[{'all': [], 'any': [{'data': {'bgColor': '#ff...","[cat.color, wcag2aa, wcag143]"
1,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures every id attribute value of active ele...,IDs of active elements must be unique,https://dequeuniversity.com/rules/axe/3.1/dupl...,duplicate-id-active,serious,"[{'all': [], 'any': [{'data': 'logo', 'id': 'd...","[cat.parsing, wcag2a, wcag411]"
2,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures the order of headings is semantically ...,Heading levels should only increase by one,https://dequeuniversity.com/rules/axe/3.1/head...,heading-order,moderate,"[{'all': [], 'any': [{'data': 3, 'id': 'headin...","[cat.semantics, best-practice]"
3,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures every form element has a label,Form elements must have labels,https://dequeuniversity.com/rules/axe/3.1/labe...,label,critical,"[{'all': [], 'any': [{'data': None, 'id': 'ari...","[cat.forms, wcag2a, wcag332, wcag131, section5..."
4,Abadia Dos Dourados (MG),310010,3100104,MG,Região Sudeste,7006,False,1,https://abadiadosdourados.mg.gov.br/novo/index...,Ensures the page has only one main landmark an...,Page must have one main landmark,https://dequeuniversity.com/rules/axe/3.1/land...,landmark-one-main,moderate,"[{'all': [{'data': None, 'id': 'page-has-main'...","[cat.semantics, best-practice]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4748,Cocal do Sul (SC),420425,4204251,SC,Região Sul,16821,False,3,https://www.diariomunicipal.sc.gov.br/site/?r=...,Ensures all page content is contained by landm...,All page content must be contained by landmarks,https://dequeuniversity.com/rules/axe/3.1/regi...,region,moderate,"[{'all': [], 'any': [{'data': None, 'id': 'reg...","[cat.keyboard, best-practice]"
4749,Cuiabá (MT),510340,5103403,MT,Região Centro-Oeste,618124,True,3,https://diariooficial.cuiaba.mt.gov.br/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4750,Guarabira (PB),250630,2506301,PB,Região Nordeste,59115,False,3,https://pmguarabira.wixsite.com/pmguarabira/di...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4751,Pará de Minas (MG),314710,3147105,MG,Região Sudeste,94808,False,3,https://transparencia.parademinas.mg.gov.br/,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
portals_violations.to_csv("a11y_analysis.csv", index=False)

In [4]:
portals_violations = pd.read_csv("a11y_analysis.csv")

## Analisar os resultados - Cidades acima de 100 mil habitantes

Nesta seção, analisamos os resultados da avaliação de acessibilidade dos **portais principais** dos municípios com 100 mil habitantes ou mais, de acordo com a população estimada pelo IBGE para o ano de 2020.

No total, foram encontradas **2230 violações às regras de acessibilidade** avaliadas, sendo **493 violações consideradas críticas** (22% das violações encontradas) e 729 graves (33% do total).

In [119]:
violations_100k_main = (
    portals_violations
    .query("populacao_2020 >= 100000 & fonte_prioridade == 1")
    .reset_index(drop=True)
)

print(
    "Total de municípios com mais de 100 mil hab. com portais analisados: ",
    len(violations_100k_main["IBGE"].unique()),
)
total_portals = len(violations_100k_main["fonte"].unique())
print("Total de portais únicos: ", total_portals)

total_violations = sum(violations_100k_main.groupby("fonte")["id"].count())
print("Total de violações encontradas: ", total_violations)

(
    violations_100k_main
    .groupby(["fonte", "impact"])["id"]
    .count()
    .groupby("impact")
    .sum()[["critical", "serious", "moderate", "minor"]]
    .rename("num_violacoes")
    .to_frame()
    .assign(percentual_violacoes=lambda _: round(
        100*_["num_violacoes"]/total_violations, 2
    ))
)

Total de municípios com mais de 100 mil hab. com portais analisados:  323
Total de portais únicos:  306
Total de violações encontradas:  2217


,num_violacoes,percentual_violacoes
impact,,
critical,493,22.24
serious,729,32.88
moderate,827,37.30
minor,168,7.58


Em média, foram encontradas, em média, 7,15 problemas de acessibilidade por município analisado. Dessas, havia, em média,:

- 1,59 violações críticas;
- 2,35 violações graves;
- 2,67 violações moderadas;
- 0,54 violações leves.

Isso significa que há uma média de 3,94 problemas críticos ou graves de acessibilidade por município cujo portal foi analisado no recorte considerado.

In [132]:
import plotly.express as px


violations_impact = (
    violations_100k_main
    .groupby([
        "municipio",
        "IBGE",
        "IBGE7",
        "UF",
        "regiao",
        "populacao_2020",
        "eh_capital",
        "impact",
    ])["id"]
    # CREDIT: https://stackoverflow.com/a/49128246/7733563
    .count()
    .unstack(fill_value=0)
    .stack()
    .rename("num_violacoes")
    .reset_index()
)


def plot_violations_hist(df: pd.DataFrame, impact_level: str="all"):
    if impact_level != "all":
        df = df.copy().query("impact == @impact_level")
    fig = px.histogram(
        df,
        x="num_violacoes",
        color="impact",
        title=f"Violation frequency - {impact_level.title()} violations",
        labels={"num_violacoes": "number of violations"},
        color_discrete_sequence=["#c60a1c", "#ff684c", "#e39802", "#ffda66"],
        category_orders={
            "impact": ["critical", "serious", "moderate", "minor"]
        },
        template="simple_white",
        range_x=(0, 6.99),
        range_y=(0, 300),
    )
    violations_avg = sum(df["num_violacoes"]) / len(df["IBGE"].unique())
    if impact_level != "all":
        annotation_x = violations_avg
        fig.add_shape(
            type="line",
            x0=violations_avg,
            x1=violations_avg,
            y0=0,
            y1=300,
            opacity=1,
            line={"width": 3, "color": "black", "dash": "dash"},
        )
    else:
        annotation_x = 4
    fig.add_annotation(
        x=annotation_x,
        y=300,
        text=f"<b>Mean: {round(violations_avg,2)}</b>",
        xref="x",
        yref="y",
        xanchor="left",
        xshift=5,
        showarrow=False,
        font={"size": 16}
    )
    return fig

plot_violations_hist(violations_impact, "critical").show()
plot_violations_hist(violations_impact, "serious").show()
plot_violations_hist(violations_impact, "moderate").show()
plot_violations_hist(violations_impact, "minor").show()
plot_violations_hist(violations_impact, "all").show()

     IBGE                municipio  num_violacoes
0  240810               Natal (RN)             16
1  311940  Coronel Fabriciano (MG)             14
2  351870             Guarujá (SP)             14
3  251370          Santa Rita (PB)             14
4  351640     Franco da Rocha (SP)             14
5  210005          Açailândia (MA)             13
6  410180           Araucária (PR)             13
7  352390                 Itu (SP)             13
8  330190            Itaboraí (RJ)             13
9  420460            Criciúma (SC)             13


Os dez municípios com maior número de problemas encontrados nos respectivos portais principais são:

1. Natal (RN)
2. Coronel Fabriciano (MG)
3. Guarujá (SP)
4. Franco da Rocha (SP)
5. Santa Rita (PB)
6. Açailândia (MA)
7. Itaboraí (RJ)
8. Criciúma (SC)
9. Itu (SP)
10. Teresópolis (RJ)

In [159]:
violations_rank = (
    violations_impact
    .pivot(
        index=[
            "IBGE",
            "regiao",
            "UF",
            "municipio",
            "eh_capital",
            "populacao_2020",
        ],
        columns="impact",
        values="num_violacoes",
    )
    .eval("total = critical + serious + moderate + minor")
    .sort_values(
        ["total", "critical", "serious", "moderate", "minor"], ascending=False
    )
    .reset_index()
    .rename_axis(columns={"impact": "posicao"})
    .eval("critical_or_serious = critical + serious")
)

violations_rank.head(10)[[
    "IBGE",
    "municipio",
    # "populacao_2020",
    "critical",
    "serious",
    "moderate",
    "minor",
    "total",
]]

posicao,IBGE,municipio,critical,serious,moderate,minor,total
0,240810,Natal (RN),5,5,4,2,16
1,311940,Coronel Fabriciano (MG),3,6,4,1,14
2,351870,Guarujá (SP),3,6,3,2,14
3,351640,Franco da Rocha (SP),3,4,5,2,14
4,251370,Santa Rita (PB),2,6,6,0,14
5,210005,Açailândia (MA),5,3,4,1,13
6,330190,Itaboraí (RJ),4,5,3,1,13
7,420460,Criciúma (SC),3,5,3,2,13
8,352390,Itu (SP),3,4,4,2,13
9,330580,Teresópolis (RJ),3,4,3,3,13


Não existe uma relação significativa entre número de violações sérias e críticas e o tamanho da população (correlação de postos de Spearman = 0,07; p=0,25).

In [191]:
from scipy.stats import spearmanr
populationXviolations_corr = spearmanr(
    violations_rank[["populacao_2020", "critical_or_serious"]]
)

print(
    "A correlação de postos entre o número de violações sérias ou críticas "
    "e o tamanho da população residente é de "
    f"{populationXviolations_corr.correlation:.2f} "
    f"(p={populationXviolations_corr.pvalue:.2f}).")

if populationXviolations_corr.pvalue > 0.05:
    print("Não é possível descartar que não haja correlação.")

px.scatter(
    violations_rank,
    x="populacao_2020",
    y="critical_or_serious",
    log_x=True,
    title="Number of serious and critical violations, versus city size",
    labels={
        "critical_or_serious": "serious and critical violations",
        "populacao_2020": "estimated population",
    },
)

A correlação de postos entre o número de violações sérias ou críticas e o tamanho da população residente é de 0.07 (p=0.25).
Não é possível descartar que não haja correlação.


As capitais analisadas possuem, em média, 4,46 erros sérios ou críticos em seus portais (mediana: 5). Os demais municípios possuem uma média um pouco menor - em média, 3,89 erros sérios ou críticos (mediana: 4) -, mas essa diferença não chega a ser estatisticamente significativa.

In [193]:
from scipy.stats import mannwhitneyu

critical_or_serious_capitals = (
    violations_rank.query("eh_capital")["critical_or_serious"]
)
critical_or_serious_noncapitals = (
    violations_rank.query("not eh_capital")["critical_or_serious"]
)

print(
    "A média de erros sérios ou críticos em capitais é de",
    f"{critical_or_serious_capitals.mean().round(2)}",
    f"(desvio-padrão: {critical_or_serious_capitals.std().round(2)})."
)

print(
    "A média de erros sérios ou críticos em municípios que não são capitais",
    f"é de {critical_or_serious_noncapitals.mean().round(2)}",
    f"(desvio-padrão: {critical_or_serious_noncapitals.std().round(2)})."
)

capitalsXnoncapitals_pvalue = (
    mannwhitneyu(critical_or_serious_capitals, critical_or_serious_noncapitals)
    .pvalue
)

if capitalsXnoncapitals_pvalue > 0.05:
    print(
        "Não é possível afirmar que as distribuições são diferentes",
        f"(p={capitalsXnoncapitals_pvalue:.2f})."
    )

px.box(
    violations_rank,
    x="eh_capital",
    y="critical_or_serious",
    points="all",
    title=(
        "Number of serious and critical violations - capitals versus "
        "non-capital cities"
    ),
    labels={
        "critical_or_serious": "serious and critical violations",
        "populacao_2020": "estimated population",
    },
)

A média de erros sérios ou críticos em capitais é de 4.46 (desvio-padrão: 1.82).
A média de erros sérios ou críticos em municípios que não são capitais é de 3.89 (desvio-padrão: 2.11).
Não é possível afirmar que as distribuições são diferentes (p=0.24).


In [199]:
violations_rank["regiao"]

0          Região Nordeste
1           Região Sudeste
2           Região Sudeste
3           Região Sudeste
4          Região Nordeste
              ...         
305             Região Sul
306    Região Centro-Oeste
307    Região Centro-Oeste
308         Região Sudeste
309         Região Sudeste
Name: regiao, Length: 310, dtype: object

As regiões com as maiores médias e medianas de problemas críticos ou graves encontrados são Sudeste (média: 4.48; mediana: 5) e Centro-Oeste (média: 4.22; mediana: 5).

Se destacam com as menores proporções de erros críticos ou graves as regiões Norte (média: 2,93; mediana: 3) e Nordeste (média: 3,47; mediana: 3).

In [225]:
from scipy.stats import kruskal

critical_or_serious_regions = list()

for region in ["Norte", "Nordeste", "Centro-Oeste", "Sudeste", "Sul"]:
    critical_or_serious_region = (
        violations_rank
        .query(
            "regiao.str.contains(@region)", engine="python"
        )["critical_or_serious"]
    )
    critical_or_serious_regions.append(critical_or_serious_region)
    print(
        f"A média de erros sérios ou críticos para a Região {region} é de "
        f"{critical_or_serious_region.mean():.2f} (mediana: "
        f"{critical_or_serious_region.median():.0f})."
    )

regions_kruskal = kruskal(*critical_or_serious_regions)

if regions_kruskal.pvalue <= 0.05:
    print(
        "É possível afirmar que pelo menos uma das regiões têm mediana "
        f"distinta das demais (p={regions_kruskal.pvalue:.1e})"
    )

px.box(
    violations_rank,
    x="regiao",
    y="critical_or_serious",
    points="all",
    title="Number of serious and critical violations per region",
    labels={
        "critical_or_serious": "serious and critical violations",
        "populacao_2020": "estimated population",
    },
)

A média de erros sérios ou críticos para a Região Norte é de 2.93 (mediana: 3).
A média de erros sérios ou críticos para a Região Nordeste é de 3.47 (mediana: 3).
A média de erros sérios ou críticos para a Região Centro-Oeste é de 4.22 (mediana: 5).
A média de erros sérios ou críticos para a Região Sudeste é de 4.48 (mediana: 5).
A média de erros sérios ou críticos para a Região Sul é de 3.39 (mediana: 4).
É possível afirmar que pelo menos uma das regiões têm mediana distinta das demais (p=6.6e-05)


### Análise das violações mais comuns no recorte

As cinco violações mais comuns são:

1. a falta de uso de seções para indicar as partes principais da página, dificultando o reconhecimento e navegação com leitor de telas;
2. a ausência de uma seção principal que indique a parte relevante do conteúdo na página, também dificultando o reconhecimento e navegação com leitor de telas;
3. o uso de combinações de cores com pouco contraste entre o texto e o fundo, dificultando a leitura por pessoas com baixa visão;
4. a ausência de legendas para elementos de formulário, dificultando fortemente o correto preenchimento por pessoas que utilizam leitores de tela;
5. links sem texto passível de visualização por leitores de tela, dificultando a navegação por usuários dessa tecnologia assistiva.

In [227]:
common_violations = (
    violations_100k_main.value_counts(["id"])
    .rename("num_occurences")
    .to_frame()
)

In [226]:
# separate descriptions for violations that occoured among the portals
violations_descr = (
    violations_100k_main
    .groupby(["id", "help", "description", "impact", "helpUrl", "tags"])
    .first()
    .reset_index(["help", "description", "impact", "helpUrl", "tags"])
)

# list violations by frequency
(
    violations_descr
    .merge(common_violations, on="id")
    [["help", "impact", "num_occurences"]]
    .sort_values("num_occurences", ascending=False)
    .head(15)
)

,help,impact,num_occurences,helpUrl
id,,,,
region,All page content must be contained by landmarks,moderate,270,https://dequeuniversity.com/rules/axe/3.1/regi...
landmark-one-main,Page must have one main landmark,moderate,263,https://dequeuniversity.com/rules/axe/3.1/land...
color-contrast,Elements must have sufficient color contrast,serious,234,https://dequeuniversity.com/rules/axe/3.1/colo...
label,Form elements must have labels,critical,165,https://dequeuniversity.com/rules/axe/3.1/labe...
label,Form elements must have labels,serious,165,https://dequeuniversity.com/rules/axe/3.1/labe...
page-has-heading-one,Page must contain a level-one heading,moderate,141,https://dequeuniversity.com/rules/axe/3.1/page...
link-name,Links must have discernible text,serious,141,https://dequeuniversity.com/rules/axe/3.1/link...
image-alt,Images must have alternate text,critical,118,https://dequeuniversity.com/rules/axe/3.1/imag...
html-has-lang,<html> element must have a lang attribute,serious,117,https://dequeuniversity.com/rules/axe/3.1/html...
